**Описание проекта**

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Наша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Задачи**:
- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

**Подключение к базе данных**

In [1]:
import pandas as pd

In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

**Исследование таблиц — вывести первые строки.** 

In [3]:
query = ''' SELECT * FROM books LIMIT 1'''
books = pd.io.sql.read_sql(query, con = engine) 
books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


In [4]:
query = ''' SELECT * FROM authors LIMIT 1'''
authors = pd.io.sql.read_sql(query, con = engine) 
authors

,author_id,author
0,1,A.S. Byatt


In [5]:
query = ''' SELECT * FROM publishers LIMIT 1'''
publishers = pd.io.sql.read_sql(query, con = engine) 
publishers

,publisher_id,publisher
0,1,Ace


In [6]:
query = ''' SELECT * FROM ratings LIMIT 1'''
ratings = pd.io.sql.read_sql(query, con = engine) 
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


In [7]:
query = ''' SELECT * FROM reviews LIMIT 1'''
reviews = pd.io.sql.read_sql(query, con = engine) 
reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


**Выполнение задач**

**1. Посчитайте, сколько книг вышло после 1 января 2000 года.**

In [8]:
query = ''' 
SELECT COUNT(book_id) 
  FROM books 
 WHERE publication_date > '2000-01-01' 
 '''
books1 = pd.io.sql.read_sql(query, con = engine) 
books1

,count
0,819


После 1 января 2000 года вышло 819 книг. 

**2. Для каждой книги посчитайте количество обзоров и среднюю оценку.**

In [9]:
query = '''
with 
a as 
(
SELECT books.book_id
     , books.title
     , COUNT(reviews.review_id) AS count_reviews
  FROM books 
LEFT JOIN reviews 
    ON books.book_id = reviews.book_id
GROUP BY(books.book_id, books.title)), 

b as 
(
SELECT books.book_id
     , books.title
     , AVG(ratings.rating) AS avg_ratings
  FROM books 
LEFT JOIN ratings 
    ON books.book_id = ratings.book_id
GROUP BY(books.book_id, books.title))

SELECT a.title
     , a.count_reviews
     , b.avg_ratings
  FROM a
INNER JOIN b on a.book_id = b.book_id 
ORDER BY a.count_reviews, avg_ratings
'''

books2 = pd.io.sql.read_sql(query, con = engine)
books2

,title,count_reviews,avg_ratings
0,The Natural Way to Draw,0,3.000000
1,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
2,Leonardo's Notebooks,0,4.000000
3,Essential Tales and Poems,0,4.000000
4,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
...,...,...,...
995,The Glass Castle,6,4.206897
996,The Book Thief,6,4.264151
997,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
998,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634


Для каждой книги мы посчитали количество обзоров и среднюю оценку. Самое большое количество обзоров (7 штук) у книги Twilight (Twilight #1).
Для книги с нулевым количеством обзоров минимальный средний рейтинг 3.0.
Для книги с максимальным количеством обзоров максимальный средний рейтинг 3.6.

**3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.**

In [10]:
query = ''' 
with a 
as
(
SELECT publishers.publisher_id
     , publisher
     , COUNT(book_id) AS count_books
  FROM books 
INNER JOIN publishers 
    ON books.publisher_id = publishers.publisher_id 
WHERE num_pages > 50
GROUP BY publisher, publishers.publisher_id)

, b as
(
SELECT max(count_books) as max_count_books
  FROM a
)
  
SELECT a.publisher_id
     , a.publisher
     , b.max_count_books
  FROM a 
INNER JOIN b
    ON 1=1
WHERE a.count_books = b.max_count_books
'''
books3 = pd.io.sql.read_sql(query, con = engine) 
books3

,publisher_id,publisher,max_count_books
0,212,Penguin Books,42


Издательство, которое выпустило наибольшее число книг толще 50 страниц -- Penguin Books.

**4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.**

In [11]:
query = ''' 
with a 
as
(
SELECT authors.author_id
     , author
     , AVG(rating) AS avg_rating
 FROM books 
LEFT JOIN ratings 
   ON books.book_id = ratings.book_id 
LEFT JOIN authors 
   ON books.author_id = authors.author_id 
GROUP BY authors.author_id, author
HAVING COUNT(rating) > 50
)

, b as
(
SELECT max(avg_rating) as max_rating
  FROM a
)

SELECT a.author_id
     , a.author
     , b.max_rating
  FROM a
INNER JOIN b
    on 1 = 1
 WHERE a.avg_rating = b.max_rating
'''
books4 = pd.io.sql.read_sql(query, con = engine) 
books4

,author_id,author,max_rating
0,236,J.K. Rowling/Mary GrandPré,4.288462


Автор с самой высокой средней оценкой книг (учитывая только книги с 50 и более оценками) -- J.K. Rowling/Mary GrandPré. 

**5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.**

In [12]:
query = ''' 
with 
a as 
(
SELECT username, COUNT(rating) AS count_rating
  FROM ratings 
GROUP BY username
HAVING COUNT(rating) > 50
)

,b as
(
SELECT reviews.username
     , count(review_id) as review_count
  FROM reviews
INNER JOIN a on a.username = reviews.username
GROUP BY reviews.username
)

SELECT avg(review_count)
FROM b
'''

books5 = pd.io.sql.read_sql(query, con = engine)
books5

,avg
0,24.333333


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок -- 24.33.

В условиях изоляции из-за коронавируса люди стали реже выходить из дома, но стали больше читать. В связи с этим наша компания решила купила крупный сервис для чтения книг по подписке. Нашей задачей было проанализировали базу данных. Наши действия: 
- Мы исследовали базу данных, вывели первую строку из всех таблиц;
- Выяснили сколько книг вышло после 1 января 2000 года (их 819);
- Для каждой книги посчитали количество обзоров и среднюю оценку. Выяснили, что самое большое количество обзоров (7 штук) у книги Twilight (Twilight #1). Для книги с нулевым количеством обзоров минимальный средний рейтинг 3.0. Для книги с максимальным количеством обзоров максимальный средний рейтинг 3.6.
- Определили издательство, которое выпустило наибольшее число книг толще 50 страниц (издательство Penguin Books);
- Определили автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками (автор -- J.K. Rowling/Mary GrandPré).
- Посчитали среднее количество обзоров от пользователей, которые поставили больше 50 оценок (среднее количество обзоров -- 24.3).